# Key papers

This Jupyter Notebook can be used to perform basic publication analysis for a science branch. 

**Features:**

1. Subtopic analysis based on co-citation graph clustering:
    * Chord diagram for co-citation graph
    * Comparison of subtopics by size
    * Timeline of each subtopic
    * Extraction of 1,2,3-grams describing each subtopic
2. Detection of highlight papers:
    * Top cited papers overall
    * Detection of most cited papers for each year
    * Detection of papers with max relative citation gain for each year
3. Citation dynamics visualization for highlight papers
4. Subtopic evolution tracking based on co-citation graph clustering for different time periods

## Getting Started

1. Define the `SEARCH_TERMS` variable in the cell below with a list of keywords that describe the science branch of your interest.
2. Run all cells & see the results.

In [ ]:
SEARCH_TERMS = ['dna', 'methylation', 'clock']

## Publication Analysis

In [ ]:
import logging

from bokeh.plotting import show, output_notebook
from matplotlib import pyplot as plt

from keypaper.pm_loader import PubmedLoader
from keypaper.analysis import KeyPaperAnalyzer
from keypaper.visualization import Plotter

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s: %(message)s')
output_notebook()
%matplotlib inline

In [ ]:
analyzer = KeyPaperAnalyzer(PubmedLoader(test=False))
log = analyzer.launch(*SEARCH_TERMS)

In [ ]:
plotter = Plotter(analyzer)

## Subtopics a.k.a. Clusters in the Co-citation Graph

In [ ]:
show(plotter.chord_diagram_components())

In [ ]:
show(plotter.component_size_summary())

In [ ]:
for p in plotter.subtopic_timeline_graphs():
    show(p)

In [ ]:
show(plotter.component_ratio())

## Top Cited Papers Overall

In [ ]:
show(plotter.top_cited_papers())

## Top Cited Papers for Each Year

In [ ]:
show(plotter.max_gain_papers())

## Top by Relative Gain for Each Year

In [ ]:
show(plotter.max_relative_gain_papers())

## Citation per Year Dynamics

In [ ]:
plotter.article_citation_dynamics()

In [ ]:
analyzer.df

# Experimental Features

## Component Evolution

In [ ]:
# plt = plotter.subtopic_evolution()
# plt.show()

In [ ]:
import math
import holoviews as hv
hv.extension('bokeh')

import pandas as pd

In [ ]:
analyzer.subtopic_evolution_analysis(step=2)
evol = plotter.analyzer.evolution_df.copy()
for col in evol:
    evol[col] = evol[col].apply(lambda x: math.floor(x))

In [ ]:
cols = evol.columns[1:]
pairs = list(zip(cols, cols[1:]))
nodes = []
edges = []

for now, then in pairs:
    nodes_now = [f'{now} {c}' for c in evol[now].unique()]
    nodes_then = [f'{then} {c}' for c in evol[then].unique()]
    inner = {node : 0 for node in nodes_then}
    changes = {node : inner.copy() for node in nodes_now}
    for pmid, comp in evol.iterrows():
        c_now, c_then = comp[now], comp[then]
        changes[f'{now} {c_now}'][f'{then} {c_then}'] += 1
    
#     nodes.extend(nodes_now)
    for v in nodes_now:
        for u in nodes_then:
            if changes[v][u] > 0:
                edges.append([v, u, changes[v][u]])
    
#     edges.append([str(now), str(then), 1])
    
# nodes.extend(nodes_then)
    
# print(nodes)

In [ ]:
sankey = hv.Sankey(edges)
sankey.opts(width=960, height=400)

## PageRank for Citation Analysis

In [ ]:
analyzer.load_citations()

In [ ]:
import networkx as nx

# Apply PageRank algorithm with damping factor of 0.5
pr_nx = nx.pagerank(analyzer.G, alpha=0.5, tol=1e-9)

In [ ]:
ancestor = dict.fromkeys(analyzer.G, (0, 0))

# Select ancestor with highest PR for each node
for v in analyzer.G:
    for u in analyzer.G[v]:
        anc, pr = ancestor[u]
        if pr_nx[v] > pr:
            ancestor[u] = (v, pr_nx[v])

In [ ]:
PRG = nx.DiGraph()
for v, anc in ancestor.items():
    u, pr = anc
    if pr > 0:
        PRG.add_edge(u, v)

In [ ]:
start, end = zip(*list(PRG.edges()))

In [ ]:
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider, Circle, HoverTool, MultiLine
from bokeh.models.graphs import NodesAndLinkedEdges

node_indices = list(filter(lambda node: len(analyzer.df[analyzer.df['pmid'] == node]) > 0, list(PRG.nodes())))

years = []
year_counts = {}
pageranks = []
size = []
for node in node_indices:
    year = analyzer.df[analyzer.df['pmid'] == node]['year'].values[0]
    if not year in year_counts:
        year_counts[year] = 1
    else:
        year_counts[year] += 1
    years.append(year)
    pageranks.append(pr_nx[node] * 100)
    size.append(pr_nx[node] * 1000)
max_year_count = max(list(year_counts.values()))
min_year, max_year = min(years), max(years)

plot = figure(title="PageRank applied to citation filtering", 
              x_range=(min_year - 1, max_year+1), y_range=(0, max_year_count + 1),
              tools="", toolbar_location=None)

plot.add_tools(HoverTool(tooltips=[("PMID", "@pmid"), ("Year", "@year"), ("PageRank", "@pagerank %")]))

graph = GraphRenderer()

graph.node_renderer.data_source.add(node_indices, 'index')
graph.node_renderer.data_source.data['pmid'] = node_indices
graph.node_renderer.data_source.data['year'] = years
graph.node_renderer.data_source.data['pagerank'] = pageranks
graph.node_renderer.data_source.data['size'] = size
graph.edge_renderer.data_source.data = dict(start=start, end=end)

### start of layout code   
x = [analyzer.df[analyzer.df['pmid'] == pmid]['year'].values[0] for pmid in node_indices]
y = []
tmp_year_counts = {}
for node in node_indices:
    year = analyzer.df[analyzer.df['pmid'] == node]['year'].values[0]
    if not year in tmp_year_counts:
        tmp_year_counts[year] = 1
    else:
        tmp_year_counts[year] += 1
    y.append(tmp_year_counts[year])

graph_layout = dict(zip(node_indices, zip(x, y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

graph.node_renderer.glyph = Circle(size='size', fill_color='blue')
graph.node_renderer.hover_glyph = Circle(size='size', fill_color='green')

graph.edge_renderer.glyph = MultiLine(line_color='black', line_alpha=1, line_width=1)
graph.edge_renderer.hover_glyph = MultiLine(line_color='green', line_width=2)

graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(graph)

show(plot)

## Custom PageRank version for testing

This section is devoted to experiments with PageRank on the basis of networkx source code.

In [ ]:
# Adopted from networkx source code
# https://networkx.github.io/documentation/networkx-1.10/_modules/networkx/algorithms/link_analysis/pagerank_alg.html#pagerank

import numpy as np

def pagerank(G, alpha=0.85, personalization=None,
             max_iter=100, tol=1.0e-6, nstart=None, weight='weight',
             dangling=None):
    """Return the PageRank of the nodes in the graph.

    PageRank computes a ranking of the nodes in the graph G based on
    the structure of the incoming links. It was originally designed as
    an algorithm to rank web pages.

    Parameters
    ----------
    G : graph
      A NetworkX graph.  Undirected graphs will be converted to a directed
      graph with two directed edges for each undirected edge.

    alpha : float, optional
      Damping parameter for PageRank, default=0.85.

    personalization: dict, optional
      The "personalization vector" consisting of a dictionary with a
      key for every graph node and nonzero personalization value for each node.
      By default, a uniform distribution is used.

    max_iter : integer, optional
      Maximum number of iterations in power method eigenvalue solver.

    tol : float, optional
      Error tolerance used to check convergence in power method solver.

    nstart : dictionary, optional
      Starting value of PageRank iteration for each node.

    weight : key, optional
      Edge data key to use as weight.  If None weights are set to 1.

    dangling: dict, optional
      The outedges to be assigned to any "dangling" nodes, i.e., nodes without
      any outedges. The dict key is the node the outedge points to and the dict
      value is the weight of that outedge. By default, dangling nodes are given
      outedges according to the personalization vector (uniform if not
      specified). This must be selected to result in an irreducible transition
      matrix (see notes under google_matrix). It may be common to have the
      dangling dict to be the same as the personalization dict.

    Returns
    -------
    pagerank : dictionary
       Dictionary of nodes with PageRank as value

    Examples
    --------
    >>> G = nx.DiGraph(nx.path_graph(4))
    >>> pr = nx.pagerank(G, alpha=0.9)

    Notes
    -----
    The eigenvector calculation is done by the power iteration method
    and has no guarantee of convergence.  The iteration will stop
    after max_iter iterations or an error tolerance of
    number_of_nodes(G)*tol has been reached.

    The PageRank algorithm was designed for directed graphs but this
    algorithm does not check if the input graph is directed and will
    execute on undirected graphs by converting each edge in the
    directed graph to two edges.

    See Also
    --------
    pagerank_numpy, pagerank_scipy, google_matrix

    References
    ----------
    .. [1] A. Langville and C. Meyer,
       "A survey of eigenvector methods of web information retrieval."
       http://citeseer.ist.psu.edu/713792.html
    .. [2] Page, Lawrence; Brin, Sergey; Motwani, Rajeev and Winograd, Terry,
       The PageRank citation ranking: Bringing order to the Web. 1999
       http://dbpubs.stanford.edu:8090/pub/showDoc.Fulltext?lang=en&doc=1999-66&format=pdf
    """
    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G

    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()
    E = W.number_of_edges()
       
    # Number of references for each node and average for graph
    NR = D.out_degree(list(D.nodes()))
    NR_avg = E * 2 / N

    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())

    if personalization is None:
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())

    if dangling is None:
        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
    
    # power iteration: make up to max_iter iterations
    n_iter = 0
    for _ in range(max_iter):
        n_iter += 1
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] / (1 + np.sqrt(NR[n]))
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]
        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            print(f'PageRank converged in {n_iter} iterations.')
            return x

    raise NetworkXError('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter)

In [ ]:
pr = pagerank(analyzer.G, alpha=0.5, tol=1e-12)

In [ ]:
sorted(pr.items(), key=lambda x: x[1], reverse=True)